** Exploration des données**

In [1]:
!pip install geopandas shapely fiona pyproj pandas networkx matplotlib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.2/17.2 MB 107.8 MB/s eta 0:00:00


In [2]:
import geopandas as gpd
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import folium
from shapely.geometry import Point

In [3]:
from google.colab import files
uploaded = files.upload()

Saving batiments (1).csv to batiments (1).csv
Saving infra.csv to infra.csv
Saving reseau_en_arbre.xlsx to reseau_en_arbre.xlsx
Saving infrastructures.shx to infrastructures.shx
Saving infrastructures.shp to infrastructures.shp
Saving infrastructures.qix to infrastructures.qix
Saving infrastructures.dbf to infrastructures.dbf
Saving infrastructures.prj to infrastructures.prj
Saving batiments.shp to batiments.shp
Saving batiments.shx to batiments.shx
Saving batiments.qix to batiments.qix
Saving batiments.prj to batiments.prj
Saving batiments.dbf to batiments.dbf
Saving batiments.cpg to batiments.cpg


In [4]:
import os
print(os.listdir())

['.config', 'infrastructures.prj', 'batiments.shp', 'batiments.qix', 'batiments.prj', 'reseau_en_arbre.xlsx', 'infrastructures.dbf', 'batiments.shx', 'batiments.dbf', 'batiments (1).csv', 'infra.csv', 'infrastructures.shx', 'batiments.cpg', 'infrastructures.qix', 'infrastructures.shp', 'sample_data']


In [5]:
# Affichage compact
pd.set_option("display.max_rows", 20)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 160)

In [7]:
# ⚠️ On force tout en string pour ne pas casser les id comme 'E000001' ou 'P007111'
df_bat   = pd.read_csv("batiments (1).csv", dtype=str)
df_infra = pd.read_csv("infra.csv", dtype=str)

# L’Excel peut avoir plusieurs feuilles ; on prend la première par défaut
xls = pd.ExcelFile("reseau_en_arbre.xlsx")
print("Feuilles disponibles dans reseau_en_arbre.xlsx :", xls.sheet_names)
df_arbre = pd.read_excel(xls, sheet_name=xls.sheet_names[0], dtype=str)

print("\nAperçus rapides:")
print("\n--- batiments.csv ---")
display(df_bat.head())
print("\n--- infra.csv ---")
display(df_infra.head())
print("\n--- reseau_en_arbre.xlsx ---")
display(df_arbre.head())


Feuilles disponibles dans reseau_en_arbre.xlsx : ['reseau_en_arbre']

Aperçus rapides:

--- batiments.csv ---


,id_batiment,type_batiment,nb_maisons
0,E000160,habitation,8
1,E000367,habitation,3
2,E000032,habitation,1
3,E000017,habitation,7
4,E000339,habitation,1



--- infra.csv ---


,id_infra,type_infra
0,P000719,aerien
1,P007118,aerien
2,P007170,fourreau
3,P007648,aerien
4,P000306,aerien



--- reseau_en_arbre.xlsx ---


,id_batiment,nb_maisons,infra_id,infra_type,longueur
0,E000001,4,P007111,infra_intacte,12.3144613356693
1,E000001,4,P007983,infra_intacte,40.3209288665353
2,E000001,4,P000308,infra_intacte,39.1407985664577
3,E000001,4,P007819,infra_intacte,17.3904643240185
4,E000002,1,P007111,infra_intacte,12.3144613356693


In [8]:
def profile(df, name):
    print(f"\n===== {name} =====")
    print("Shape :", df.shape)
    print("Colonnes :", list(df.columns))
    print("\nTypes (pandas dtypes) :")
    print(df.dtypes)
    print("\nValeurs manquantes par colonne :")
    print(df.isna().sum())
    print("\nLignes entièrement dupliquées :", df.duplicated().sum())

profile(df_bat,   "batiments.csv")
profile(df_infra, "infra.csv")
profile(df_arbre, "reseau_en_arbre.xlsx")



===== batiments.csv =====
Shape : (381, 3)
Colonnes : ['id_batiment', 'type_batiment', 'nb_maisons']

Types (pandas dtypes) :
id_batiment      object
type_batiment    object
nb_maisons       object
dtype: object

Valeurs manquantes par colonne :
id_batiment      0
type_batiment    0
nb_maisons       0
dtype: int64

Lignes entièrement dupliquées : 0

===== infra.csv =====
Shape : (644, 2)
Colonnes : ['id_infra', 'type_infra']

Types (pandas dtypes) :
id_infra      object
type_infra    object
dtype: object

Valeurs manquantes par colonne :
id_infra      0
type_infra    0
dtype: int64

Lignes entièrement dupliquées : 0

===== reseau_en_arbre.xlsx =====
Shape : (6107, 5)
Colonnes : ['id_batiment', 'nb_maisons', 'infra_id', 'infra_type', 'longueur']

Types (pandas dtypes) :
id_batiment    object
nb_maisons     object
infra_id       object
infra_type     object
longueur       object
dtype: object

Valeurs manquantes par colonne :
id_batiment    0
nb_maisons     0
infra_id       0
infra_type

In [9]:
# On strip les espaces parasites dans toutes les colonnes texte
def strip_all(df):
    return df.apply(lambda s: s.str.strip() if s.dtype == 'object' else s)

df_bat   = strip_all(df_bat)
df_infra = strip_all(df_infra)
df_arbre = strip_all(df_arbre)


In [10]:
# Bâtiments
print("\n=== Bâtiments : colonnes attendues (ex.) id_batiment, type_batiment, nb_maisons ===")
if "id_batiment" in df_bat.columns:
    print("Bâtiments uniques :", df_bat["id_batiment"].nunique())
    print("Top 10 id_batiment :", df_bat["id_batiment"].head(10).tolist())
if "type_batiment" in df_bat.columns:
    print("\nRépartition type_batiment :")
    display(df_bat["type_batiment"].value_counts(dropna=False).head(20))
if "nb_maisons" in df_bat.columns:
    print("\nStat sur nb_maisons (si numérique) :")
    with pd.option_context('mode.use_inf_as_na', True):
        display(pd.to_numeric(df_bat["nb_maisons"], errors="coerce").describe())

# Infrastructures
print("\n=== Infrastructures : colonnes attendues (ex.) id_infra, type_infra ===")
if "id_infra" in df_infra.columns:
    print("Infras uniques :", df_infra["id_infra"].nunique())
    print("Top 10 id_infra :", df_infra["id_infra"].head(10).tolist())
if "type_infra" in df_infra.columns:
    print("\nRépartition type_infra :")
    display(df_infra["type_infra"].value_counts(dropna=False))

# Réseau logique
print("\n=== Réseau logique : colonnes attendues (ex.) id_batiment, infra_id, infra_type, longueur ===")
for col in ["id_batiment","infra_id","infra_type","longueur","nb_maisons"]:
    if col in df_arbre.columns:
        nun = df_arbre[col].nunique() if col != "longueur" else "—"
        print(f"{col}: uniques={nun}")
if "infra_type" in df_arbre.columns:
    print("\nRépartition infra_type (logique) :")
    display(df_arbre["infra_type"].value_counts(dropna=False).head(20))
if "longueur" in df_arbre.columns:
    print("\nStats sur longueur (si numérique) :")
    display(pd.to_numeric(df_arbre["longueur"], errors="coerce").describe())
if "nb_maisons" in df_arbre.columns:
    print("\nStats sur nb_maisons (réseau) :")
    display(pd.to_numeric(df_arbre["nb_maisons"], errors="coerce").describe())



=== Bâtiments : colonnes attendues (ex.) id_batiment, type_batiment, nb_maisons ===
Bâtiments uniques : 381
Top 10 id_batiment : ['E000160', 'E000367', 'E000032', 'E000017', 'E000339', 'E000121', 'E000136', 'E000131', 'E000298', 'E000324']

Répartition type_batiment :


,count
type_batiment,
habitation,379
hôpital,1
école,1



Stat sur nb_maisons (si numérique) :


/tmp/ipython-input-4106465240.py:11: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):


,nb_maisons
count,381.000000
mean,4.375328
std,2.253260
min,1.000000
25%,2.000000
50%,4.000000
75%,6.000000
max,8.000000


...

,count
infra_type,
infra_intacte,5530
a_remplacer,577



Stats sur longueur (si numérique) :


,longueur
count,6107.000000
mean,29.314081
std,21.681575
min,0.372684
25%,8.476727
50%,27.771443
75%,49.956115
max,76.985593



Stats sur nb_maisons (réseau) :


,nb_maisons
count,6107.000000
mean,1.016211
std,0.203714
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,4.000000


In [11]:
# 5.1 — Les id_batiment du réseau existent-ils dans batiments.csv ?
if {"id_batiment"} <= set(df_arbre.columns) and "id_batiment" in df_bat.columns:
    ids_arbre = set(df_arbre["id_batiment"].dropna().unique())
    ids_bats  = set(df_bat["id_batiment"].dropna().unique())
    not_in_bat = ids_arbre - ids_bats
    not_in_arbre = ids_bats - ids_arbre
    print("\nBÂTIMENTS — présents dans réseau mais pas dans batiments.csv :", len(not_in_bat))
    print("BÂTIMENTS — présents dans batiments.csv mais absents du réseau :", len(not_in_arbre))

# 5.2 — Les infra_id du réseau existent-ils dans infra.csv ?
if "infra_id" in df_arbre.columns and "id_infra" in df_infra.columns:
    infra_ids_arbre = set(df_arbre["infra_id"].dropna().unique())
    infra_ids_tab   = set(df_infra["id_infra"].dropna().unique())
    infra_missing_typo = infra_ids_arbre - infra_ids_tab
    infra_unused       = infra_ids_tab - infra_ids_arbre
    print("\nINFRAS — utilisées par le réseau mais sans enregistrement dans infra.csv :", len(infra_missing_typo))
    print("INFRAS — présentes dans infra.csv mais non référencées par le réseau :", len(infra_unused))

# 5.3 — Duplications suspectes côté réseau (mêmes paires batiment/infra)
if {"id_batiment","infra_id"} <= set(df_arbre.columns):
    dupl_pairs = df_arbre.duplicated(subset=["id_batiment","infra_id"]).sum()
    print("\nRÉSEAU — paires (id_batiment, infra_id) dupliquées :", dupl_pairs)



BÂTIMENTS — présents dans réseau mais pas dans batiments.csv : 0
BÂTIMENTS — présents dans batiments.csv mais absents du réseau : 0

INFRAS — utilisées par le réseau mais sans enregistrement dans infra.csv : 0
INFRAS — présentes dans infra.csv mais non référencées par le réseau : 0

RÉSEAU — paires (id_batiment, infra_id) dupliquées : 521


In [12]:
# Échantillons et listings utiles (facultatif)
df_bat.sample(min(50, len(df_bat))).to_csv("inspect_batiments_sample.csv", index=False)
df_infra.sample(min(50, len(df_infra))).to_csv("inspect_infra_sample.csv", index=False)
df_arbre.sample(min(50, len(df_arbre))).to_csv("inspect_reseau_sample.csv", index=False)

# Listes des IDs orphelins (si présents)
if "id_batiment" in df_bat.columns and "id_batiment" in df_arbre.columns:
    pd.Series(sorted(list(not_in_bat))).to_csv("ids_reseau_sans_batiments.csv", index=False, header=["id_batiment"])
    pd.Series(sorted(list(not_in_arbre))).to_csv("ids_batiments_sans_reseau.csv", index=False, header=["id_batiment"])

if "infra_id" in df_arbre.columns and "id_infra" in df_infra.columns:
    pd.Series(sorted(list(infra_missing_typo))).to_csv("ids_infras_sans_typologie.csv", index=False, header=["infra_id"])
    pd.Series(sorted(list(infra_unused))).to_csv("ids_infras_non_referencees.csv", index=False, header=["id_infra"])

print("\nFichiers d’inspection exportés (si non vides) :")
print("- inspect_batiments_sample.csv")
print("- inspect_infra_sample.csv")
print("- inspect_reseau_sample.csv")
print("- ids_reseau_sans_batiments.csv")
print("- ids_batiments_sans_reseau.csv")
print("- ids_infras_sans_typologie.csv")
print("- ids_infras_non_referencees.csv")



Fichiers d’inspection exportés (si non vides) :
- inspect_batiments_sample.csv
- inspect_infra_sample.csv
- inspect_reseau_sample.csv
- ids_reseau_sans_batiments.csv
- ids_batiments_sans_reseau.csv
- ids_infras_sans_typologie.csv
- ids_infras_non_referencees.csv


### **le traimenet analytique **

In [14]:
# Chargement
df_bat = pd.read_csv("batiments (1).csv", dtype=str)
df_infra = pd.read_csv("infra.csv", dtype=str)
df_arbre = pd.read_excel("reseau_en_arbre.xlsx", dtype=str)

# Nettoyage léger
for df in [df_bat, df_infra, df_arbre]:
    df.columns = df.columns.str.strip()
    df = df.apply(lambda s: s.str.strip() if s.dtype == "object" else s)

# Conversion de colonnes numériques
df_bat["nb_maisons"] = pd.to_numeric(df_bat["nb_maisons"], errors="coerce")
df_arbre["nb_maisons"] = pd.to_numeric(df_arbre["nb_maisons"], errors="coerce")
df_arbre["longueur"] = pd.to_numeric(df_arbre["longueur"], errors="coerce")

print("✅ Chargement terminé :")
print(" - Bâtiments :", df_bat.shape)
print(" - Infras :", df_infra.shape)
print(" - Réseau logique :", df_arbre.shape)

✅ Chargement terminé :
 - Bâtiments : (381, 3)
 - Infras : (644, 2)
 - Réseau logique : (6107, 5)


→ on ajoute la typologie physique (aérien, semi-aérien, fourreau).

In [15]:
df_join = df_arbre.merge(
    df_infra,
    left_on="infra_id",
    right_on="id_infra",
    how="left",
    validate="many_to_one"
)

print("🚧 Données après jointure :")
print(df_join[["infra_id","type_infra","infra_type","longueur"]].head())


🚧 Données après jointure :
  infra_id type_infra     infra_type   longueur
0  P007111   fourreau  infra_intacte  12.314461
1  P007983     aerien  infra_intacte  40.320929
2  P000308   fourreau  infra_intacte  39.140799
3  P007819   fourreau  infra_intacte  17.390464
4  P007111   fourreau  infra_intacte  12.314461


Correction du nombre de prises

In [16]:
df_join = df_join.drop(columns=["nb_maisons"], errors="ignore")
df_join = df_join.merge(
    df_bat[["id_batiment","nb_maisons"]],
    on="id_batiment",
    how="left",
    validate="many_to_one"
)

Ajout des coûts et durées unitaires

In [17]:
cout_par_m = {"aerien":500, "semi-aerien":750, "fourreau":900}
duree_par_m = {"aerien":2, "semi-aerien":4, "fourreau":5}

df_join["cout_m"] = df_join["type_infra"].map(cout_par_m)
df_join["duree_h_m"] = df_join["type_infra"].map(duree_par_m)

df_join["cout_total"] = df_join["cout_m"] * df_join["longueur"]
df_join["duree_totale_h"] = df_join["duree_h_m"] * df_join["longueur"]

print("💰 Coûts et durées ajoutés :")
display(df_join.head(5))


💰 Coûts et durées ajoutés :


,id_batiment,infra_id,infra_type,longueur,id_infra,type_infra,nb_maisons,cout_m,duree_h_m,cout_total,duree_totale_h
0,E000001,P007111,infra_intacte,12.314461,P007111,fourreau,3,900,5,11083.015202,61.572307
1,E000001,P007983,infra_intacte,40.320929,P007983,aerien,3,500,2,20160.464433,80.641858
2,E000001,P000308,infra_intacte,39.140799,P000308,fourreau,3,900,5,35226.718710,195.703993
3,E000001,P007819,infra_intacte,17.390464,P007819,fourreau,3,900,5,15651.417892,86.952322
4,E000002,P007111,infra_intacte,12.314461,P007111,fourreau,3,900,5,11083.015202,61.572307


In [18]:
agg_infra = (
    df_join.groupby("type_infra")
    .agg(
        n_troncons=("infra_id","nunique"),
        longueur_totale_m=("longueur","sum"),
        cout_total_euro=("cout_total","sum"),
        duree_totale_h=("duree_totale_h","sum"),
        cout_moyen_par_m=("cout_m","mean"),
    )
)
agg_infra["cout_total_k€"] = agg_infra["cout_total_euro"]/1000
display(agg_infra)


,n_troncons,longueur_totale_m,cout_total_euro,duree_totale_h,cout_moyen_par_m,cout_total_k€
type_infra,,,,,,
aerien,369,94755.539718,4.737777e+07,189511.079436,500.0,47377.769859
fourreau,197,78803.403933,7.092306e+07,394017.019666,900.0,70923.063540
semi-aerien,78,5462.148234,4.096611e+06,21848.592934,750.0,4096.611175


Bâtiments prioritaires : effort cumulé

In [19]:
agg_bat = (
    df_join.groupby("id_batiment")
    .agg(
        longueur_totale=("longueur","sum"),
        cout_total=("cout_total","sum"),
        duree_totale_h=("duree_totale_h","sum"),
        nb_maisons=("nb_maisons","first")
    )
)
agg_bat["cout_moyen_par_prise"] = agg_bat["cout_total"] / agg_bat["nb_maisons"].clip(lower=1)
display(agg_bat.describe())


,longueur_totale,cout_total,duree_totale_h,nb_maisons,cout_moyen_par_prise
count,381.000000,381.000000,381.000000,381.000000,381.000000
mean,469.871632,321253.135365,1588.915202,4.375328,114051.905558
std,202.457925,125895.888061,596.548911,2.253260,113166.441555
min,79.170312,49436.725148,232.227392,1.000000,11426.118332
25%,271.817055,202789.554353,1053.522345,2.000000,44086.495817
50%,487.731040,324632.996882,1616.896486,4.000000,71808.611903
75%,631.537910,430200.654566,2158.895644,6.000000,133286.435435
max,810.140997,529001.635138,2599.907195,8.000000,527306.105686


Détection préliminaire des nœuds (structure du graphe)

In [20]:
df_nodes = (
    df_join.groupby("infra_id")
    .agg(
        n_bat_desservis=("id_batiment","nunique"),
        type_infra=("type_infra","first"),
        longueur=("longueur","mean"),
        cout_total=("cout_total","sum")
    )
    .sort_values("n_bat_desservis", ascending=False)
)

df_nodes["is_noeud"] = df_nodes["n_bat_desservis"] > 1
n_noeuds = df_nodes["is_noeud"].sum()
print(f"🔍 Nombre estimé de nœuds de mutualisation : {n_noeuds}")
display(df_nodes.head(10))


🔍 Nombre estimé de nœuds de mutualisation : 190


,n_bat_desservis,type_infra,longueur,cout_total,is_noeud
infra_id,,,,,
P007111,381,fourreau,12.314461,4.588368e+06,True
P007982,337,fourreau,49.956115,1.515169e+07,True
P000181,322,aerien,6.189343,9.964842e+05,True
P000394,321,fourreau,37.518306,1.083904e+07,True
P000279,226,aerien,50.202139,7.003198e+06,True
P000290,226,aerien,8.476727,9.833003e+05,True
P000292,226,aerien,7.445240,1.206129e+06,True
P000046,226,aerien,8.304525,1.158481e+06,True
P000317,226,aerien,54.690026,8.941819e+06,True


**Visualisation des types de Batiments & Infrastructures à remplacer**

In [23]:
# Tables attributaires
df_bat   = pd.read_csv("batiments (1).csv", dtype=str)
df_infra = pd.read_csv("infra.csv", dtype=str)
df_arbre = pd.read_excel("reseau_en_arbre.xlsx", dtype=str)

# Shapefiles géométriques
gdf_infra = gpd.read_file("infrastructures.shp")   # colonnes attendues: infra_id, longueur, geometry (LINESTRING)
gdf_bat   = gpd.read_file("batiments.shp")        # colonnes attendues: id_batiment, geometry (POINT)


In [24]:
for df in (df_bat, df_infra, df_arbre):
    df.columns = df.columns.str.strip()
    for c in df.columns:
        if df[c].dtype == object:
            df[c] = df[c].str.strip()

# Numériques
df_bat["nb_maisons"]    = pd.to_numeric(df_bat["nb_maisons"], errors="coerce")
df_arbre["nb_maisons"]  = pd.to_numeric(df_arbre["nb_maisons"], errors="coerce")  # sera écrasé par fichier batiments
df_arbre["longueur"]    = pd.to_numeric(df_arbre["longueur"],   errors="coerce")

# Harmonise les CRS en WGS84 pour Folium si besoin
try:
    if gdf_infra.crs is not None and gdf_infra.crs.to_epsg() != 4326:
        gdf_infra = gdf_infra.to_crs(4326)
    if gdf_bat.crs is not None and gdf_bat.crs.to_epsg() != 4326:
        gdf_bat = gdf_bat.to_crs(4326)
except Exception as e:
    print("⚠️ CRS non défini dans un des shapefiles, projection inchangée.")


In [25]:
# Jointure réseau logique ↔ typologie infra physique
df_join = df_arbre.merge(
    df_infra, left_on="infra_id", right_on="id_infra", how="left", validate="many_to_one"
)

# Remplace nb_maisons réseau par la réf ‘batiments.csv’ (source de vérité)
df_join = df_join.drop(columns=["nb_maisons"], errors="ignore").merge(
    df_bat[["id_batiment","nb_maisons","type_batiment"]],
    on="id_batiment", how="left", validate="many_to_one"
)

# Barèmes coût & durée
COUT_M   = {"aerien":500, "semi-aerien":750, "fourreau":900}
DUREE_MH = {"aerien":2,   "semi-aerien":4,   "fourreau":5}

df_join["cout_m"]       = df_join["type_infra"].map(COUT_M).fillna(0)
df_join["duree_h_m"]    = df_join["type_infra"].map(DUREE_MH).fillna(0)
df_join["cout_total"]   = df_join["cout_m"]    * df_join["longueur"]
df_join["duree_totale"] = df_join["duree_h_m"] * df_join["longueur"]


In [26]:
# Mutualisation par tronçon (combien de bâtiments / prises sur un même infra)
agg_infra = (
    df_join.groupby(["infra_id","type_infra"], as_index=False)
           .agg(
               bat_desservis=("id_batiment","nunique"),
               prises_total=("nb_maisons","sum"),
               longueur_ref=("longueur","mean"),
               cout_total=("cout_total","sum"),
               duree_totale=("duree_totale","sum"),
               infra_type_logique=("infra_type","first")  # ex: infra_intacte / a_remplacer
           )
)

# Difficulté indicative par tronçon: longueur / prises (anti-division par 0)
agg_infra["prises_total"] = agg_infra["prises_total"].fillna(0)
agg_infra["difficulte_infra"] = agg_infra["longueur_ref"] / agg_infra["prises_total"].where(agg_infra["prises_total"]>0, 1)

# Nœud potentiel = tronçon partagé par >=2 bâtiments
agg_infra["is_noeud"] = agg_infra["bat_desservis"] >= 2

# Join attributaire sur le shapefile des lignes
gdf_infra_enrich = gdf_infra.merge(
    agg_infra, left_on="infra_id", right_on="infra_id", how="left", validate="one_to_one"
)


In [27]:
print("Columns in gdf_bat:", gdf_bat.columns)
gdf_bat_enrich = gdf_bat.merge(
    df_bat[["id_batiment","type_batiment","nb_maisons"]],
    left_on="id_bat", right_on="id_batiment", how="left", validate="one_to_one"
)

Columns in gdf_bat: Index(['id_bat', 'nb_maisons', 'geometry'], dtype='object')


In [29]:
# Centre carte
if not gdf_bat_enrich.empty:
    center = [gdf_bat_enrich.geometry.y.mean(), gdf_bat_enrich.geometry.x.mean()]
else:
    # fallback sur bbox infra
    cx = gdf_infra_enrich.geometry.centroid.x.mean()
    cy = gdf_infra_enrich.geometry.centroid.y.mean()
    center = [cy, cx]

m = folium.Map(location=center, zoom_start=13, tiles="CartoDB positron")

# Palette lignes par type_infra
LINE_COLORS = {"aerien":"#1f77b4", "semi-aerien":"#ff7f0e", "fourreau":"#2ca02c"}
def style_line(feat):
    props = feat["properties"]
    t = props.get("type_infra", "")
    base_color = LINE_COLORS.get(t, "#888888")
    # surcouche rouge si a_remplacer
    bad = str(props.get("infra_type_logique","")).lower() == "a_remplacer"
    color = "#d62728" if bad else base_color
    w = 3 if bad else 2
    return {"color": color, "weight": w, "opacity": 0.9}

# Lignes
folium.GeoJson(
    gdf_infra_enrich.to_json(),
    name="Infrastructures",
    style_function=style_line,
    tooltip=folium.GeoJsonTooltip(
        fields=["infra_id","type_infra","infra_type_logique","bat_desservis","prises_total","longueur_ref","cout_total","difficulte_infra"],
        aliases=["Infra","Type","État logique","Bât. desservis","Prises tot.","Longueur (m)","Coût (€)","Difficulté (m/prise)"],
        localize=True
    )
).add_to(m)

# Points bâtiment (couleur par type)
def color_bat(row):
    tb = (row.get("type_batiment") or "").lower()
    if "hôpital" in tb or "hopital" in tb:
        return "red"
    if "école" in tb or "ecole" in tb:
        return "orange"
    return "blue"

for _, r in gdf_bat_enrich.iterrows():
    folium.CircleMarker(
        location=[r.geometry.y, r.geometry.x],
        radius=5,
        color=color_bat(r),
        fill=True, fill_opacity=0.9,
        popup=f"{r['id_batiment']} · {r.get('type_batiment','?')} · {r.get('nb_maisons','?')} prises"
    ).add_to(m)

'''# “Nœuds” = tronçons mutualisés (>=2 bâtiments) → on place un marqueur au centroïde
noeuds = gdf_infra_enrich[gdf_infra_enrich["is_noeud"]==True].copy()
noeuds["centroid"] = noeuds.geometry.centroid
for _, r in noeuds.iterrows():
    c = r["centroid"]
    folium.CircleMarker(
        location=[c.y, c.x],
        radius=6, color="purple", fill=True, fill_opacity=0.8,
        popup=f"NOEUD · {r['infra_id']} · {int(r.get('bat_desservis',0))} bâtiments"
    ).add_to(m)
'''
folium.LayerControl().add_to(m)
m
